In [57]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tqdm.notebook as tqdm

In [58]:
df_hcahps = pd.read_csv("HCAHPS-Hospital.csv")

/var/folders/fq/6j5vjdy13w36prs570mvsk300000gn/T/ipykernel_79561/1639603615.py:1: DtypeWarning: Columns (12,14,17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_hcahps = pd.read_csv("HCAHPS-Hospital.csv")


In [59]:
df_hcahps.head()

,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,HCAHPS Measure ID,HCAHPS Question,...,Patient Survey Star Rating Footnote,HCAHPS Answer Percent,HCAHPS Answer Percent Footnote,HCAHPS Linear Mean Value,Number of Completed Surveys,Number of Completed Surveys Footnote,Survey Response Rate Percent,Survey Response Rate Percent Footnote,Start Date,End Date
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,H_COMP_1_A_P,"Patients who reported that their nurses ""Alway...",...,NaN,72,NaN,Not Applicable,643,NaN,17,NaN,01/01/2023,12/31/2023
1,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,H_COMP_1_SN_P,"Patients who reported that their nurses ""Somet...",...,NaN,6,NaN,Not Applicable,643,NaN,17,NaN,01/01/2023,12/31/2023
2,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,H_COMP_1_U_P,"Patients who reported that their nurses ""Usual...",...,NaN,22,NaN,Not Applicable,643,NaN,17,NaN,01/01/2023,12/31/2023
3,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,H_COMP_1_LINEAR_SCORE,Nurse communication - linear mean score,...,NaN,Not Applicable,NaN,89,643,NaN,17,NaN,01/01/2023,12/31/2023
4,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,H_COMP_1_STAR_RATING,Nurse communication - star rating,...,NaN,Not Applicable,NaN,Not Applicable,643,NaN,17,NaN,01/01/2023,12/31/2023


In [60]:
df_hcahps["Facility ID"].unique()

array(['010001', '010005', '010006', ..., '670323', '671300', '671301'],
      dtype=object)

In [61]:
(df_hcahps["Facility Name"].str.lower().str.contains("New York-Presbyterian Hospital".lower())).sum()

93

In [62]:
questions = df_hcahps["HCAHPS Question"].unique()
questions

array(['Patients who reported that their nurses "Always" communicated well',
       'Patients who reported that their nurses "Sometimes" or "Never" communicated well',
       'Patients who reported that their nurses "Usually" communicated well',
       'Nurse communication - linear mean score',
       'Nurse communication - star rating',
       'Patients who reported that their nurses "Always" treated them with courtesy and respect',
       'Patients who reported that their nurses "Sometimes" or "Never"  treated them with courtesy and respect',
       'Patients who reported that their nurses "Usually"  treated them with courtesy and respect',
       'Patients who reported that their nurses "Always" listened carefully to them',
       'Patients who reported that their nurses "Sometimes" or "Never" listened carefully to them',
       'Patients who reported that their nurses "Usually" listened carefully to them',
       'Patients who reported that their nurses "Always" explained things in

In [63]:
summaries = [q for q in questions if "-" in q]
summaries

['Nurse communication - linear mean score',
 'Nurse communication - star rating',
 'Doctor communication - linear mean score',
 'Doctor communication - star rating',
 'Staff responsiveness - linear mean score',
 'Staff responsiveness - star rating',
 'Communication about medicines - linear mean score',
 'Communication about medicines - star rating',
 'Discharge information - linear mean score',
 'Discharge information - star rating',
 'Care transition - linear mean score',
 'Care transition - star rating',
 'Cleanliness - linear mean score',
 'Cleanliness - star rating',
 'Quietness - linear mean score',
 'Quietness - star rating',
 'Overall hospital rating - linear mean score',
 'Overall hospital rating - star rating',
 'Recommend hospital - linear mean score',
 'Recommend hospital - star rating']

In [64]:
import re

def get_stats(hospital_data):
    hospital_data = hospital_data.reset_index(drop=True).set_index("HCAHPS Question")
    questions_cleaned = {}

    for q in questions:
        if q in summaries:
            continue

        if q not in hospital_data.index:
            continue
            
        if "always" not in q.lower() and "never" not in q.lower() and "usually" not in q.lower() and "sometimes" not in q.lower() and "rarely" not in q.lower() and "never" not in q.lower():
            print(q)
            if "NO" in q:
                name = "".join(q.split("NO"))
                if name not in questions_cleaned:
                    questions_cleaned[name] = {}
                questions_cleaned[name]["NO"] = hospital_data.loc[q, "HCAHPS Answer Percent"]
            if "YES" in q:
                name = "".join(q.split("YES"))
                if name not in questions_cleaned:
                    questions_cleaned[name] = {}
                questions_cleaned[name]["YES"] = hospital_data.loc[q, "HCAHPS Answer Percent"]
            if "ree" in q:
                name = "".join(q.split('"')[:1]+q.split('"')[-1:])
                inside = "".join(q.split('"')[1:-1])

                name = re.sub(r"\s+", " ", name).strip()
                
                if name not in questions_cleaned:
                    questions_cleaned[name] = {}
                
                questions_cleaned[name][inside] = hospital_data.loc[q, "HCAHPS Answer Percent"]
            if "rating" in q:
                pass
            continue
        
        name = "".join(q.split('"')[:1]+q.split('"')[-1:])
        inside = "".join(q.split('"')[1:-1])

        name = re.sub(r"\s+", " ", name).strip()
        
        if name not in questions_cleaned:
            questions_cleaned[name] = {}
        
        questions_cleaned[name][inside] = hospital_data.loc[q, "HCAHPS Answer Percent"]
        
    return questions_cleaned

In [65]:
stats = get_stats(df_hcahps[df_hcahps["Facility ID"] == df_hcahps["Facility ID"].unique()[3]])
stats

Patients who reported that NO, they were not given information about what to do during their recovery at home
Patients who reported that YES, they were given information about what to do during their recovery at home
Patients who reported that NO, they did not discuss whether they would need help after discharge
Patients who reported that YES, they did discuss whether they would need help after discharge
Patients who reported that NO, they did not receive written information about possible symptoms to look out for after discharge
Patients who reported that YES, they did receive written information about possible symptoms to look out for after discharge
Patients who "Agree" they understood their care when they left the hospital
Patients who "Disagree" or "Strongly Disagree" they understood their care when they left the hospital
Patients who "Strongly Agree" they understood their care when they left the hospital
Patients who "Agree" that the staff took my preferences into account when de

{'Patients who reported that their nurses communicated well': {'Always': '80',
  'Sometimes or Never': '1',
  'Usually': '19'},
 'Patients who reported that their nurses treated them with courtesy and respect': {'Always': '85',
  'Sometimes or Never': '2',
  'Usually': '13'},
 'Patients who reported that their nurses listened carefully to them': {'Always': '76',
  'Sometimes or Never': '1',
  'Usually': '23'},
 'Patients who reported that their nurses explained things in a way they could understand': {'Always': '78',
  'Sometimes or Never': '0',
  'Usually': '22'},
 'Patients who reported that their doctors communicated well': {'Always': '84',
  'Sometimes or Never': '1',
  'Usually': '15'},
 'Patients who reported that their doctors treated them with courtesy and respect': {'Always': '90',
  'Sometimes or Never': '1',
  'Usually': '9'},
 'Patients who reported that their doctors listened carefully to them': {'Always': '81',
  'Sometimes or Never': '3',
  'Usually': '16'},
 'Patients w

In [66]:
def get_summary_stats(hospital_data):
    hospital_data = hospital_data.reset_index(drop=True).set_index("HCAHPS Question")
    questions_cleaned = {}

    for q in summaries:
        if "star" in q.lower():
            continue

        lmv = hospital_data.loc[q]["HCAHPS Linear Mean Value"]
        questions_cleaned[q.split("-")[0].removesuffix(" ")] = lmv



    return questions_cleaned

In [67]:
df_hcahps_ny = df_hcahps[df_hcahps["State"] == "NY"]
df_hcahps_ny["Facility Name"].unique()

array(['KALEIDA HEALTH', "ST JOSEPH'S MEDICAL CENTER",
       'WYOMING COUNTY COMMUNITY HOSPITAL', 'BRONXCARE HOSPITAL CENTER',
       'OUR LADY OF LOURDES MEMORIAL HOSPITAL, INC',
       'ALBANY MEDICAL CENTER HOSPITAL',
       'JAMAICA HOSPITAL MEDICAL CENTER',
       'NEW YORK COMMUNITY HOSPITAL OF BROOKLYN, INC.',
       'VASSAR BROTHERS MEDICAL CENTER', 'MOUNT SINAI HOSPITAL',
       'NASSAU UNIVERSITY MEDICAL CENTER',
       'RICHMOND UNIVERSITY MEDICAL CENTER',
       'NEWARK-WAYNE COMMUNITY HOSPITAL', 'CHENANGO MEMORIAL HOSPITAL',
       'NS/LIJ HS SOUTHSIDE HOSPITAL', 'WYNN HOSPITAL',
       'NS/LIJ HS HUNTINGTON HOSPITAL',
       "MOUNT SINAI ST LUKE'S ROOSEVELT HOSPITAL", "ST MARY'S HEALTHCARE",
       'NORTHERN DUTCHESS HOSPITAL', 'NEW YORK-PRESBYTERIAN/QUEENS',
       'BROOKLYN HOSPITAL CENTER - DOWNTOWN CAMPUS',
       "ST PETER'S HOSPITAL", 'GENEVA GENERAL HOSPITAL',
       'MONTEFIORE MEDICAL CENTER',
       'NIAGARA FALLS MEMORIAL MEDICAL CENTER',
       'UNITED MEMORI

In [68]:
hospital_stats_ny = {}

for hid in tqdm.tqdm(df_hcahps_ny["Facility ID"].unique()):
    hospital_stats_ny[hid] = {
        "id": hid,
        "name": df_hcahps_ny[df_hcahps_ny["Facility ID"] == hid]["Facility Name"].unique()[0],
        "satisfaction_stats": get_stats(df_hcahps_ny[df_hcahps_ny["Facility ID"] == hid]),
        "satisfaction_summary_stats": get_summary_stats(df_hcahps_ny[df_hcahps_ny["Facility ID"] == hid])
    }

  0%|          | 0/163 [00:00<?, ?it/s]

Patients who reported that NO, they were not given information about what to do during their recovery at home
Patients who reported that YES, they were given information about what to do during their recovery at home
Patients who reported that NO, they did not discuss whether they would need help after discharge
Patients who reported that YES, they did discuss whether they would need help after discharge
Patients who reported that NO, they did not receive written information about possible symptoms to look out for after discharge
Patients who reported that YES, they did receive written information about possible symptoms to look out for after discharge
Patients who "Agree" they understood their care when they left the hospital
Patients who "Disagree" or "Strongly Disagree" they understood their care when they left the hospital
Patients who "Strongly Agree" they understood their care when they left the hospital
Patients who "Agree" that the staff took my preferences into account when de

In [69]:
hospital_stats_ny

{'330005': {'id': '330005',
  'name': 'KALEIDA HEALTH',
  'satisfaction_stats': {'Patients who reported that their nurses communicated well': {'Always': '71',
    'Sometimes or Never': '6',
    'Usually': '23'},
   'Patients who reported that their nurses treated them with courtesy and respect': {'Always': '79',
    'Sometimes or Never': '4',
    'Usually': '17'},
   'Patients who reported that their nurses listened carefully to them': {'Always': '67',
    'Sometimes or Never': '8',
    'Usually': '25'},
   'Patients who reported that their nurses explained things in a way they could understand': {'Always': '68',
    'Sometimes or Never': '7',
    'Usually': '25'},
   'Patients who reported that their doctors communicated well': {'Always': '72',
    'Sometimes or Never': '6',
    'Usually': '22'},
   'Patients who reported that their doctors treated them with courtesy and respect': {'Always': '79',
    'Sometimes or Never': '5',
    'Usually': '16'},
   'Patients who reported that thei

In [70]:
import json

In [71]:
with open('hospital_base_stats.json', 'w+') as f:
    json.dump(hospital_stats_ny, f)

In [32]:
with open('hospital_costs.json', 'r') as f:
    hospital_costs = json.load(f)

print(hospital_costs)

{'4.0': {'name': 'Albany Memorial Hospital', 'costs_by_condition_severity': {'Cardiac Conditions': {'1': {'cost': 3585.05, 'ranking': 'average'}, '2': {'cost': 5253.15, 'ranking': 'below average'}, '3': {'cost': 9068.1, 'ranking': 'average'}, '4': {'cost': 15230.62, 'ranking': 'average'}}, 'Other': {'1': {'cost': 7533.43, 'ranking': 'average'}, '2': {'cost': 12709.99, 'ranking': 'average'}, '3': {'cost': 3734.21, 'ranking': 'below average'}, '4': {'cost': 104436.29, 'ranking': 'significantly above average'}}, 'Neurological Disorders': {'1': {'cost': 3745.02, 'ranking': 'average'}, '2': {'cost': 7880.32, 'ranking': 'above average'}, '3': {'cost': 9293.42, 'ranking': 'average'}, '4': {'cost': 76964.49, 'ranking': 'significantly above average'}}, 'Cancer-Related Conditions': {'1': {'cost': 4818.69, 'ranking': 'average'}, '2': {'cost': 13312.8, 'ranking': 'above average'}, '3': {'cost': 6711.04, 'ranking': 'below average'}, '4': {'cost': 11535.19, 'ranking': 'below average'}}, 'Infectious 

In [33]:
hospital_costs = {v["name"]: v for v in hospital_costs.values()}
hospital_costs

{'Albany Memorial Hospital': {'name': 'Albany Memorial Hospital',
  'costs_by_condition_severity': {'Cardiac Conditions': {'1': {'cost': 3585.05,
     'ranking': 'average'},
    '2': {'cost': 5253.15, 'ranking': 'below average'},
    '3': {'cost': 9068.1, 'ranking': 'average'},
    '4': {'cost': 15230.62, 'ranking': 'average'}},
   'Other': {'1': {'cost': 7533.43, 'ranking': 'average'},
    '2': {'cost': 12709.99, 'ranking': 'average'},
    '3': {'cost': 3734.21, 'ranking': 'below average'},
    '4': {'cost': 104436.29, 'ranking': 'significantly above average'}},
   'Neurological Disorders': {'1': {'cost': 3745.02, 'ranking': 'average'},
    '2': {'cost': 7880.32, 'ranking': 'above average'},
    '3': {'cost': 9293.42, 'ranking': 'average'},
    '4': {'cost': 76964.49, 'ranking': 'significantly above average'}},
   'Cancer-Related Conditions': {'1': {'cost': 4818.69, 'ranking': 'average'},
    '2': {'cost': 13312.8, 'ranking': 'above average'},
    '3': {'cost': 6711.04, 'ranking': 'be

In [45]:
reg = re.compile( "[A-Z ]{6,}[A-Z ]{6,}")

In [52]:
j = 0
for hid in hospital_stats_ny:
    name = hospital_stats_ny[hid]["name"].lower()
    name = "".join([a for a in name if a in "qwertyuiopasdfghjklzxcvbnm "])
    added = False
    for hospital_name in hospital_costs:
        hn = "".join([a for a in hospital_name.lower() if a in "qwertyuiopasdfghjklzxcvbnm "])
        if name in hn or hn in name:
            hospital_stats_ny[hid]["costs"] = hospital_costs[hospital_name]["costs_by_condition_severity"]
            added = True

    if not added:
        hospital_stats_ny[hid]["costs"] = {}
        j += 1
        print("failed to find costs for", hospital_stats_ny[hid]["name"])
print(j)

failed to find costs for KALEIDA HEALTH
failed to find costs for BRONXCARE HOSPITAL CENTER
failed to find costs for WYNN HOSPITAL
failed to find costs for ADIRONDACK MEDICAL CENTER - SARANAC LAKE
failed to find costs for JONES MEMORIAL HOSPITAL
failed to find costs for ALBANY VA MEDICAL CENTER
failed to find costs for NY EYE AND EAR INFIRMARY OF MOUNT SINAI
failed to find costs for NEW YORK-PRESBYTERIAN HOSPITAL
failed to find costs for NYACK HOSPITAL
failed to find costs for ONEIDA HEALTH HOSPITAL
failed to find costs for BATH VA MEDICAL CENTER
failed to find costs for GARNET HEALTH MEDICAL CENTER
failed to find costs for UPSTATE NEW YORK VA HEALTHCARE SYSTEM (WESTERN NY VA HEALTHCARE SYSTEM)
failed to find costs for BASSETT HEALTHCARE
failed to find costs for LONG ISLAND COMMUNITY HOSPITAL
failed to find costs for VA HUDSON VALLEY HEALTHCARE SYSTEM
failed to find costs for BRONX VA MEDICAL CENTER
failed to find costs for GUTHRIE CORTLAND REGIONAL MEDICAL CENTER
failed to find costs f

In [55]:
hospital_stats_ny["330014"]["costs"]

{'Neurological Disorders': {'1': {'cost': 2794.18, 'ranking': 'below average'},
  '2': {'cost': 3738.87, 'ranking': 'average'},
  '3': {'cost': 5510.85, 'ranking': 'below average'},
  '4': {'cost': 23637.44, 'ranking': 'average'}},
 'Respiratory Conditions': {'1': {'cost': 4449.14, 'ranking': 'average'},
  '2': {'cost': 5788.75, 'ranking': 'average'},
  '3': {'cost': 6914.33, 'ranking': 'average'},
  '4': {'cost': 11201.17, 'ranking': 'average'}},
 'Cardiac Conditions': {'1': {'cost': 2438.06, 'ranking': 'below average'},
  '2': {'cost': 24873.64, 'ranking': 'significantly above average'},
  '3': {'cost': 24080.95, 'ranking': 'above average'},
  '4': {'cost': 65013.62, 'ranking': 'significantly above average'}},
 'Other': {'1': {'cost': 15094.07, 'ranking': 'average'},
  '2': {'cost': 31606.03, 'ranking': 'above average'},
  '3': {'cost': 79151.14, 'ranking': 'significantly above average'},
  '4': {'cost': 120961.67, 'ranking': 'significantly above average'}},
 'Trauma & Injury': {'1':

In [56]:
with open("hospital_stats_ny_with_costs.json", "w+") as f:
    json.dump(hospital_stats_ny, f)